In [15]:
import sys
sys.path.append(r'./20200327')
sys.path.append(r'D:\\python_code\\data_explore')
import numpy as np
import utils
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import population_dyn as popd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from IPython.display import display

In [30]:
def update_trace(a, b, c):
    delt_lam = a
    k = b
    phi_0 = c
    y = popd.ratio_pop((delt_lam, k, phi_0), t)
    fig_pop.data[0].y = y
    fig_pop.layout.barmode = 'overlay'
    
    
def update_expdata(a):
    select = a
    get_datatag = ['#%d-1' % select, '#%d-2' % select]
    fig_pop.data[1].x = data.loc[data['Sample']==get_datatag[0], :]['Time']
    fig_pop.data[1].y = data.loc[data['Sample']==get_datatag[0], :]['mCherry_ratio']
    fig_pop.data[2].x = data.loc[data['Sample']==get_datatag[1], :]['Time']
    fig_pop.data[2].y = data.loc[data['Sample']==get_datatag[1], :]['mCherry_ratio']

def output_pars(a, b, c):
    display(r'k: %.4f, $\Delta \lambda$: $%.4f, $\phi_A$: %.4f' % (a, b, c))

# slid1 for k
def updat_sld_k(*args):
    sld_k.max = text_k.value
    sld_k.min = - text_k.value

sld_k = widgets.FloatSlider(value=0.001, min=-0.1, max=0.1, step=0.00002, 
                           description='k')
text_k = widgets.FloatText(value=0.1)
c_1 = widgets.HBox([sld_k, text_k])
text_k.observe(updat_sld_k, 'value')
# slid2 for delta_l
def updat_sld_delta_l(*args):
    sld_delta_l.max = text_delta_l.value
    sld_delta_l.min =  - text_delta_l.value

sld_delta_l = widgets.FloatSlider(value=0.003, min=-0.1, max=0.1, step=0.00002, 
                           description='$\Delta \lambda$')
text_delta_l = widgets.FloatText(value=0.1)
c_2 = widgets.HBox([sld_delta_l, text_delta_l])
text_delta_l.observe(updat_sld_delta_l, 'value')
# slid for phi
sld_phi = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.001, 
                           description='$\phi_A$')


# trace for phi

t = np.linspace(0, 50, num=100)
trace_ratio = go.Scatter(x=t, y=popd.ratio_pop((sld_delta_l.value, sld_k.value, sld_phi.value), t),
                         name = 'ratio_simulation', line=dict(color='royalblue', width=4))
fig_pop = go.FigureWidget(data=[trace_ratio],  layout=go.Layout(title={'text':'Population level'}, barmode='overlay'))
fig_pop.update_yaxes(range=[0, 1.])
updates = widgets.interactive(update_trace, a=sld_delta_l, b=sld_k, c=sld_phi)

# select data

data = pd.read_csv(r'./20200327_ratio_sum.csv')
shift = 6
data = data.loc[data['Time']>= shift]
data['Time'] = data['Time'] - shift
data_options = [i+1 for i in range(5)]
conc_list = ['0.24', '0.12', '0.9', '0.06', '0.00']
selections = [(conc_list[selc-1], selc) for selc in data_options]
drop_data = widgets.Dropdown(options = selections,
                             value=1, 
                             description='aTc conc.')

select = 1
get_datatag = ['#%d-1' % select, '#%d-2' % select]
trace_data1 = go.Scatter(x=data.loc[data['Sample']==get_datatag[0], :]['Time'], 
                         y=data.loc[data['Sample']==get_datatag[0], :]['mCherry_ratio'],
                         mode='markers', name='exp1')
trace_data2 = go.Scatter(x=data.loc[data['Sample']==get_datatag[1], :]['Time'], 
                         y=data.loc[data['Sample']==get_datatag[1], :]['mCherry_ratio'],
                         mode='markers', name='exp2')
fig_pop.add_trace(trace_data1)
fig_pop.add_trace(trace_data2)

inter_expdata = widgets.interactive(update_expdata, a=drop_data)
# output parameters
outs = widgets.interactive_output(output_pars, {'a':sld_k, 'b':sld_delta_l, 'c':sld_phi})


# c4 for all sliders
c4 = widgets.VBox([c_1, c_2, sld_phi, drop_data, outs])

c5 = widgets.VBox([c4, fig_pop])
display(c5)



In [2]:
fig, ax = plt.subplots(1,1)
sns.scatterplot(x=[0.00, 0.06, 0.09, 0.12, 0.24], y=[0.0009, 0.0035, 0.0076, 0.055, 1.216], ax=ax)
plt.ylim((0.0001, 1.4))
ax.set_ylabel('$k (h^{-1})$ ')
ax.set_xlabel('aTc conc. (ng/mL)')
ax.set_yscale('log')
plt.show()

NameError: name 'plt' is not defined